The yt clump finder was initially described in 
http://adsabs.harvard.edu/abs/2009ApJ...691..441S , but it's changed 
since then.  What it does now is decompose into non-overlapping tiles 
(stored in a kd-tree), identify contours within a tile, and then 
connect them across tiles.  It does this with an upper and lower bound 
on a field value, and looking for topologically connected sets. 

With yt, connected sets can be identified. This enables the location and analysis of **a hierarchy of
clumps**; in star formation, for instance, this allows the construction of diagrams **describing the density
at which fragmentation occurs**.

In [ ]:
import numpy as np

import yt
from yt.analysis_modules.level_sets.api import *

ds = yt.load("output/output_00028/info_00028.txt")

In [ ]:
!cat output/output_00028/*csv

In [ ]:
center = [0.53103, 0.51031000000000004, 0.50402000000000002]
region_size = 0.0015
up_vector = [0.10255487134299716, 0.059509123032244614, 0.99294569974382518]

distance = 0.00075
far_cut_depth = 0.00075

data_source = ds.sphere(center, region_size)

In [ ]:
ds.current_time, ds.current_redshift

In [ ]:
data_source.get_field_parameter?

In [ ]:
print(data_source(["density"]))

sp = data_source

# Print things in a more human-friendly manner: one density at a time
print("(x,  y,  z) Density")
print("-----------------------")
for i in range(sp["density"].size):
    print("(%f,  %f,  %f)    %f" %
          (sp["x"][i], sp["y"][i], sp["z"][i], sp["density"][i]))

In [ ]:
xx = ds.all_data()

In [ ]:
xx['density']

In [ ]:
data_source.get_data?

# assuming it's disk

In [ ]:
ds.disk?
# center, normal, radius, height, 

In [ ]:
from yt.analysis_modules.level_sets.api import Clump, find_clumps, get_lowest_clumps
    
data_source = ds.disk(center, up_vector, region_size, (far_cut_depth + distance))

c_min = 10**np.floor(np.log10(data_source['density']).min()  )
c_max = 10**np.floor(np.log10(data_source['density']).max()+1)

master_clump = Clump(data_source, 'density')
master_clump.add_validator("min_cells", 20)

find_clumps(master_clump, c_min, c_max, 2.0)
leaf_clumps = get_lowest_clumps(master_clump)



In [ ]:
prj = yt.ProjectionPlot(ds, 'z', 'density', center='c', width=(20, 'kpc'))
prj.annotate_clumps(leaf_clumps)
prj.save('clumps')

#### If the RAMSES AMR mesh isn't too highly refined or the region of interest isn't too huge, one way to get it to work would be to interpolate the data using a covering_grid or arbitrary_grid yt data object to a uniform resolution grid. 

Then reload the uniform resolution data using the yt.load_uniform_grid function, and run the clump finder on the reloaded data.


In [ ]:
ds.max_level

In [ ]:
ds.min_level

### Q: Why is the min_level non-zero? or just 1?

A: starting point non-zero or 1.

### Q: Also, why isn't max_level 17 (from the info_...txt file)? 

levelmin    =          8
levelmax    =         17

In [ ]:
dims=ds.domain_dimensions
dims

In [ ]:
ds.covering_grid?        # with no interpolation or smoothing

In [ ]:
# level:  The resolution level data to which data will be gridded. Level 0 is the root grid dx for that dataset.
cube_lvl0 = ds.covering_grid(level=0,  #  sampling just the lowest level data
                        left_edge=[0,0.0,0.0],
                        dims=ds.domain_dimensions)

In [ ]:
print(cube_lvl0['density'].shape)
print(cube_lvl0['density'][128,128,128])
# del cube_lvl0

Create a covering grid that spans two child grids of a single parent grid (with no interpolation or smoothing).

Where it is covered only by the parent grid, the cells from the parent grid will be duplicated (appropriately) to fill the covering grid.

In [ ]:
# level = 2
# dims = ds.domain_dimensions * ds.refine_by**level

cube_lvl2 = ds.covering_grid(level=2, # grid that spans two child grids of a single parent grid
                        left_edge=[0,0.0,0.0],
                        dims=ds.domain_dimensions * 2**2)   # increase the resolution of our output array by a factor of 2^2 in each direction


In [ ]:
print(cube_lvl2['density'].shape)
print(cube_lvl2['density'][512,512,512])
del cube_lvl2

There are two different types of covering grids: unsmoothed and smoothed. Smoothed grids will be filled through a cascading interpolation process; they will be filled at level 0, interpolated to level 1, filled at level 1, interpolated to level 2, filled at level 2, etc. This will help to reduce edge effects. Unsmoothed covering grids will not be interpolated, but rather values will be duplicated multiple times.

In [ ]:
# sample our dataset from above with a smoothed covering grid in order to reduce edge effects
cube_lvl2_s = ds.smoothed_covering_grid(2, [0.0, 0.0, 0.0,],
                                       ds.domain_dimensions * 2**2)

print(cube_lvl2_s['density'].shape)

# look at density at central location
print(cube_lvl2_s['density'][512,512,512])


In [ ]:
import matplotlib.pyplot as plt
plt.hist(np.log10(cube_lvl2_s['density'].flatten()))
plt.show();

In [ ]:
plt.hist(np.log10(cube_lvl0['density'].flatten()))
plt.show();

Take the level 0 (coarsest) covering grid to test out clump find function in yt.

In [ ]:
from yt.analysis_modules.level_sets.api import *

In [ ]:
# field to be used for contouring
field = ("density")        # test w/ "density" for now.

# multiplicative interval between contours.
step = 2.0

# set some sane min/max values between which we want to find contours.
# so that it won't look for
# contours connected below or above these threshold values.
c_min = 10**np.floor(np.log10(cube_lvl0[field]).min()  )
c_max = 10**np.floor(np.log10(cube_lvl0[field]).max()+1)

# Now find our 'base' clump -- this "base clump" just covers the whole domain.
master_clump = Clump(cube_lvl0, field)

# Add a "validator" to weed out clumps with less than 20 cells.
# As many validators can be added as you want.
master_clump.add_validator("min_cells", 20)

# Calculate center of mass for all clumps.
master_clump.add_info_item("center_of_mass")

# Begin clump finding.
find_clumps(master_clump, c_min, c_max, step)

# Save the clump tree as a reloadable dataset
fn = master_clump.save_as_dataset(fields=["density", "particle_mass"])

# We can traverse the clump hierarchy to get a list of all of the 'leaf' clumps
leaf_clumps = get_lowest_clumps(master_clump)

# If you'd like to visualize these clumps, a list of clumps can be supplied to
# the "clumps" callback on a plot.  First, we create a projection plot:
prj = yt.ProjectionPlot(ds, 2, field, center='c', width=(20,'kpc'))

# Next we annotate the plot with contours on the borders of the clumps
prj.annotate_clumps(leaf_clumps)

# Save the plot to disk.
prj.save('clumps')

# Reload the clump dataset.
cds = yt.load(fn)

# Clump annotation can also be done with the reloaded clump dataset.

# Remove the original clump annotation
prj.annotate_clear()

# Get the leaves and add the callback.
leaf_clumps_reloaded = cds.leaves
prj.annotate_clumps(leaf_clumps_reloaded)
prj.save('clumps_reloaded')

# Query fields for clumps in the tree.
print (cds.tree["clump", "center_of_mass"])
print (cds.tree.children[0]["grid", "density"])
print (cds.tree.children[1]["all", "particle_mass"])

# Get all of the leaf clumps.
print (cds.leaves)
print (cds.leaves[0]["clump", "cell_mass"])
